In [1]:
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

num_samples = 0 # data sample count
time_steps = 500  # Number of time steps in each sample
num_sensors = 1   # Number of sensors

# Define the full file paths for your files
data_file = 'Our_Data/sensor1_data.txt'

label_file = "ur_Data/label_data.txt"

In [2]:
# Initialize an empty list to store data for each channel
input_data = []
input_labels = []

# Load data from each text file and append it to channel_data
# for filename in data_files:
with open(data_file, 'r') as file:
    lines = file.readlines()
    channel_values = [list(map(float, line.strip().split())) for line in lines]
    input_data.append(channel_values)

x = np.array(input_data).transpose(1, 2, 0)

# Load labels from the label file
with open(label_file, 'r') as file:
    lines = file.readlines()
    input_labels = [int(value) for line in lines for value in line.strip().split()]

y_labels = np.array(input_labels)

y = to_categorical(y_labels-1,num_classes=2)  # One-hot encode the numerical labels

In [3]:
# check the data in right shape
print(x.shape)
print(y_labels.shape)

(100, 200, 6)
(100,)


In [4]:
# Split the data into training and test sets
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [4]:
# Building the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(time_steps, num_sensors)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # 2 different people

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [5]:
# Training the model
model.fit(x, y, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.5508 - loss: 15.7953 - val_accuracy: 0.0000e+00 - val_loss: 68.4108
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6047 - loss: 26.7164 - val_accuracy: 0.9500 - val_loss: 0.4722
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5648 - loss: 18.4215 - val_accuracy: 0.4000 - val_loss: 9.8876
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6703 - loss: 13.9957 - val_accuracy: 0.0000e+00 - val_loss: 39.6710
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6547 - loss: 15.2632 - val_accuracy: 0.0000e+00 - val_loss: 24.0785
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8188 - loss: 6.9925 - val_accuracy: 0.3500 - val_loss: 8.8068
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6406 - loss: 6.3727 - val_accuracy: 0.2500 - val_loss: 11.0822
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7180 - loss: 5.6142 - val_accurac

In [7]:
model.save('trained_model.keras') # Save the trained model

In [1]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.utils import to_categorical

model = load_model('trained_model.keras') # Load the trained model

In [2]:
# Define the full file paths for your files
test_data_files = [
    'sensor1_test.txt',
    'sensor2_test.txt',
    'sensor3_test.txt',
    'sensor4_test.txt',
    'sensor5_test.txt',
    'sensor6_test.txt'
]

test_label_file = "label_test.txt"

# Initialize an empty list to store data for each channel
input_test_data = []
input_test_labels = []

# Load data from each text file and append it to channel_data
for filename in test_data_files:
    with open(filename, 'r') as file:
        lines = file.readlines()
        channel_values = [list(map(float, line.strip().split())) for line in lines]
        input_test_data.append(channel_values)

x_test = np.array(input_test_data).transpose(1, 2, 0)

# Load labels from the label file
with open(test_label_file, 'r') as file:
    lines = file.readlines()
    input_test_labels = [int(value) for line in lines for value in line.strip().split()]

y_test_labels = np.array(input_test_labels)

y_test = to_categorical(y_test_labels-1,num_classes=2)  # One-hot encode the numerical labels

# print(x_test)
# print(y_test)

In [3]:
loss, accuracy = model.evaluate(x_test, y_test)
y_pred = model.predict(x_test)
y_pred_binary = np.argmax(y_pred, axis=1)
y_true_binary = np.argmax(y_test, axis=1)
print(y_true_binary)
print(y_pred_binary)

print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.6923 - loss: 1.3594
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 1 1 1]
Test Loss: 1.3594, Test Accuracy: 0.6923
